In [ ]:
# 한국사회학 1964~2025 연구논문 제목 크롤링 (연도/호수 상대탐색 + stale 방지)
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# ===== 설정 =====
SAVE_PATH = r"~~kssa_1964_2025.csv"
START_YEAR = 2025
END_YEAR = 1964  # 포함

# 드라이버
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 12)

def wait_vois_list():
    # 발행호 리스트 컨테이너 보장
    wait.until(EC.presence_of_element_located((By.ID, "voisList")))

def open_year_block(year: int):
    """특정 연도 filterGroup을 열고 그 WebElement 반환"""
    # 해당 연도 블록 찾기
    group_xpath = f'//div[@id="voisList"]/div[contains(@class,"filterGroup")][.//a[contains(@class,"btnToggleP") and contains(.,"{year}")]]'
    group = wait.until(EC.presence_of_element_located((By.XPATH, group_xpath)))

    # 연도 토글 링크
    toggle = group.find_element(By.XPATH, './/a[contains(@class,"btnToggleP")]')
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", toggle)
    time.sleep(0.2)

    # 열려있지 않으면 클릭
    cls = toggle.get_attribute("class") or ""
    if "open" not in cls:
        driver.execute_script("arguments[0].click();", toggle)
        time.sleep(0.6)

    return group  # 이후 이 그룹 내부에서만 탐색

def get_issue_links_in_group(group_we):
    """연도 그룹 내부에서 호수 <a> 요소 리스트(WebElement) 반환"""
    # 이 연도 그룹의 첫 번째 subjectSort UL
    ul = group_we.find_element(By.XPATH, './/ul[contains(@class,"subjectSort")]')
    links = ul.find_elements(By.XPATH, './li/a')
    return links

def wait_issue_loaded(expected_issue_text: str, timeout=8):
    """권호 클릭 후 헤더에 권호명이 반영될 때까지 대기"""
    # '- 제59집 제2호' -> '제59집 제2호' 로 정규화
    label = expected_issue_text.replace("-", "").strip()
    try:
        wait2 = WebDriverWait(driver, timeout)
        wait2.until(EC.text_to_be_present_in_element((By.CSS_SELECTOR, '.listHead .allNum'), label))
    except Exception:
        time.sleep(1.0)  # 느릴 때 대비

def collect_titles_from_page(year_str: str, issue_text: str):
    """현재 화면의 연구논문 제목만 수집"""
    res = []
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    vois = soup.find("div", {"id": "voisNodeList"})
    if not vois:
        return res

    current_section = None
    for child in vois.children:
        if getattr(child, "name", None) == "p" and "session" in (child.get("class") or []):
            current_section = child.get_text(strip=True)
            continue

        if getattr(child, "name", None) == "article" and "thesis" in (child.get("class") or []):
            if current_section == "연구논문":
                h2 = child.find("h2", class_="thesis__tit")
                if h2:
                    title = h2.get_text(strip=True)
                    print("   -", title)
                    res.append({"연도": year_str, "권호": issue_text, "제목": title})
    return res

# ===== 실행 =====
driver.get("https://www.dbpia.co.kr/journal/publicationDetail?publicationId=PLCT00001040")
wait_vois_list()

results = []

for year in range(START_YEAR, END_YEAR - 1, -1):
    try:
        print(f"\n===== {year}년 시작 =====")
        # 연도 블록 열기 (항상 새로 찾음)
        group = open_year_block(year)

        # 호수 링크 목록 (연도 블록 내부 상대 탐색)
        issue_links = get_issue_links_in_group(group)
        print(f"{year}년 호수 개수: {len(issue_links)}")

        for idx in range(len(issue_links)):
            # stale 방지: 매 루프마다 연도 블록과 링크를 다시 찾음
            group = open_year_block(year)
            issue_links = get_issue_links_in_group(group)
            link = issue_links[idx]

            issue_text = link.text.strip()
            if not issue_text:
                # 텍스트가 비어있으면 다시 시도
                issue_text = link.get_attribute("textContent").strip()

            print(f"\n▶ {issue_text} 크롤링 시작")

            # 권호 클릭
            driver.execute_script("arguments[0].click();", link)
            # 권호 헤더 반영 대기
            wait_issue_loaded(issue_text, timeout=10)

            # 연구논문만 수집
            year_str = str(year)
            results.extend(collect_titles_from_page(year_str, issue_text))

    except Exception as e:
        print(f"⚠ {year}년 처리 중 오류:", e)
        continue

# CSV 저장
with open(SAVE_PATH, "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.DictWriter(f, fieldnames=["연도", "권호", "제목"])
    writer.writeheader()
    writer.writerows(results)

print(f"\n✅ 저장 완료: {SAVE_PATH}")

driver.quit()